In [1]:
import pandas as pd
import kaggle


code4ML_path = "/home/ryounis/Documents/Zurich/PEACHLab/data/Code4ML"

kernels_meta = pd.read_csv(f"{code4ML_path}/kernels_meta.csv")
markup_data = pd.read_csv(f"{code4ML_path}/markup_data.csv")
vertices = pd.read_csv(f"{code4ML_path}/vertices.csv")
code_blocks = pd.read_csv(f"{code4ML_path}/code_blocks.csv")
data_preds = pd.read_csv(f"{code4ML_path}/data_with_preds.csv")

code_blocks

,code_blocks_index,kernel_id,code_block_id,code_block
0,0,17493583,0,# This Python 3 environment comes with many he...
1,1,17493583,1,import os\nimport zipfile\nfrom subprocess imp...
2,2,17493583,2,"def reduce_mem_usage(df):\n """""" iterate thr..."
3,3,17493583,3,import numpy as np
4,4,17493583,4,orders_df = reduce_mem_usage(pd.read_csv('../w...
...,...,...,...,...
2599348,2599348,8902213,22,"f=[]\n\nfor root, dirs, files in os.walk(""../o..."
2599349,2599349,8902213,23,my_model.fit_generator(\n train_data_ge...
2599350,2599350,8902213,24,test_generator.reset()\n\npred = my_model.pred...
2599351,2599351,8902213,25,import cv2\n\n\nfrom matplotlib import pyplot ...


In [2]:
kernels_count = kernels_meta.groupby('comp_name').size()
n_kernels_per_comp = kernels_meta['comp_name'].value_counts()
N_HIGHEST_COMP = 1
top_comp = kernels_count.nlargest(N_HIGHEST_COMP).index[N_HIGHEST_COMP-1]
print(f"Competition with most kernels: '{top_comp}'  ({kernels_count.nlargest(N_HIGHEST_COMP).iloc[N_HIGHEST_COMP-1]} kernels)\n")
for name in n_kernels_per_comp.index:
    print(f"{name}: {n_kernels_per_comp[name]} kernels")

Competition with most kernels: 'Titanic - Machine Learning from Disaster'  (1963 kernels)

Titanic - Machine Learning from Disaster: 1963 kernels
Digit Recognizer: 1300 kernels
House Prices - Advanced Regression Techniques: 1013 kernels
Natural Language Processing with Disaster Tweets: 725 kernels
Quora Insincere Questions Classification: 679 kernels
Mechanisms of Action (MoA) Prediction: 660 kernels
COVID19 Global Forecasting (Week 4): 434 kernels
Santander Customer Transaction Prediction: 418 kernels
PUBG Finish Placement Prediction (Kernels Only): 394 kernels
Tweet Sentiment Extraction: 386 kernels
Cassava Leaf Disease Classification: 383 kernels
Kannada MNIST: 362 kernels
Aerial Cactus Identification: 345 kernels
SIIM-ISIC Melanoma Classification: 325 kernels
Riiid Answer Correctness Prediction: 322 kernels
IEEE-CIS Fraud Detection: 303 kernels
APTOS 2019 Blindness Detection: 303 kernels
Predict Future Sales: 295 kernels
COVID19 Global Forecasting (Week 2): 288 kernels
Global Wheat

In [3]:
kernels = kernels_meta[kernels_meta['comp_name'] == top_comp]
top_comp_code_blocks = code_blocks[code_blocks['kernel_id'].isin(kernels['kernel_id'])]
merged_df = pd.merge(top_comp_code_blocks, kernels_meta, on='kernel_id')
top_comp_code_blocks = merged_df.drop(columns=['kaggle_score', 'kaggle_comments', 'kaggle_upvotes', 'comp_name'])
top_comp_code_blocks


,code_blocks_index,kernel_id,code_block_id,code_block,kernel_link
0,7019,21768463,0,# This Python 3 environment comes with many he...,/nisharahysmith/notebookda32490206
1,7020,21768463,1,"train_data = pd.read_csv(""/kaggle/input/titani...",/nisharahysmith/notebookda32490206
2,7021,21768463,2,"test_data = pd.read_csv(""/kaggle/input/titanic...",/nisharahysmith/notebookda32490206
3,7022,21768463,3,women = train_data.loc[train_data.Sex == 'fema...,/nisharahysmith/notebookda32490206
4,7023,21768463,4,men = train_data.loc[train_data.Sex == 'male']...,/nisharahysmith/notebookda32490206
...,...,...,...,...,...
76485,2547403,10882513,5,# clf = RandomForestClassifier()\nclf = Linear...,/hs1592/adkkhn-krj-titanic
76486,2547404,10882513,6,"_ = clf.fit(x,y)",/hs1592/adkkhn-krj-titanic
76487,2547405,10882513,7,pred_valid = clf.predict(x_valid)\naccuracy_sc...,/hs1592/adkkhn-krj-titanic
76488,2547406,10882513,8,pred = clf.predict(x_test),/hs1592/adkkhn-krj-titanic


In [4]:
merged_df = pd.merge(top_comp_code_blocks.merge(data_preds, on='code_blocks_index'), vertices, left_on='predicted_graph_vertex_id', right_on='graph_vertex_id')
merged_df.drop(['code_blocks_index', 'predicted_graph_vertex_id'], axis=1, inplace=True)
merged_df

,kernel_id,code_block_id,code_block,kernel_link,predicted_graph_vertex__probability,graph_vertex_id,graph_vertex_class,graph_vertex_subclass
0,21768463,0,# This Python 3 environment comes with many he...,/nisharahysmith/notebookda32490206,0.999220,88,Debug,list_files
1,21768463,1,"train_data = pd.read_csv(""/kaggle/input/titani...",/nisharahysmith/notebookda32490206,0.999642,45,Data_Extraction,load_from_csv
2,21768463,2,"test_data = pd.read_csv(""/kaggle/input/titanic...",/nisharahysmith/notebookda32490206,0.999662,45,Data_Extraction,load_from_csv
3,21768463,3,women = train_data.loc[train_data.Sex == 'fema...,/nisharahysmith/notebookda32490206,0.459239,77,Other,define_variables
4,21768463,4,men = train_data.loc[train_data.Sex == 'male']...,/nisharahysmith/notebookda32490206,0.590113,77,Other,define_variables
...,...,...,...,...,...,...,...,...
76485,10882513,5,# clf = RandomForestClassifier()\nclf = Linear...,/hs1592/adkkhn-krj-titanic,0.977052,76,Debug,commented
76486,10882513,6,"_ = clf.fit(x,y)",/hs1592/adkkhn-krj-titanic,0.999666,7,Model_Train,train_model
76487,10882513,7,pred_valid = clf.predict(x_valid)\naccuracy_sc...,/hs1592/adkkhn-krj-titanic,0.992896,48,Model_Evaluation,predict_on_test
76488,10882513,8,pred = clf.predict(x_test),/hs1592/adkkhn-krj-titanic,0.994685,48,Model_Evaluation,predict_on_test


In [5]:
merged_df.loc[merged_df['graph_vertex_class'] == 'EDA', 'graph_vertex_class'] = 'Exploratory_Data_Analysis'
merged_df.loc[merged_df['graph_vertex_class'] == 'Hyperparam_Tuning', 'graph_vertex_class'] = 'Model_Train'
merged_df.loc[merged_df['graph_vertex_class'] == 'Model_Interpretation', 'graph_vertex_class'] = 'Visualization'
merged_df.loc[merged_df['graph_vertex_class'] == 'Debug', 'graph_vertex_class'] = 'Exploratory_Data_Analysis'
merged_df.loc[merged_df['graph_vertex_class'] == 'Other', 'graph_vertex_class'] = 'Exploratory_Data_Analysis'
merged_df.loc[merged_df['graph_vertex_class'] == 'Environment', 'graph_vertex_class'] = 'Imports_and_Environment'

LABELS = [
    "Data_Transform",
    "Data_Extraction",
    "Visualization",
    "Model_Train",
    "Model_Evaluation",
    "Imports_and_Environment",
    "Data_Export",
    "Exploratory_Data_Analysis", 
    # "Other" # "Other"
]

merged_df

,kernel_id,code_block_id,code_block,kernel_link,predicted_graph_vertex__probability,graph_vertex_id,graph_vertex_class,graph_vertex_subclass
0,21768463,0,# This Python 3 environment comes with many he...,/nisharahysmith/notebookda32490206,0.999220,88,Exploratory_Data_Analysis,list_files
1,21768463,1,"train_data = pd.read_csv(""/kaggle/input/titani...",/nisharahysmith/notebookda32490206,0.999642,45,Data_Extraction,load_from_csv
2,21768463,2,"test_data = pd.read_csv(""/kaggle/input/titanic...",/nisharahysmith/notebookda32490206,0.999662,45,Data_Extraction,load_from_csv
3,21768463,3,women = train_data.loc[train_data.Sex == 'fema...,/nisharahysmith/notebookda32490206,0.459239,77,Exploratory_Data_Analysis,define_variables
4,21768463,4,men = train_data.loc[train_data.Sex == 'male']...,/nisharahysmith/notebookda32490206,0.590113,77,Exploratory_Data_Analysis,define_variables
...,...,...,...,...,...,...,...,...
76485,10882513,5,# clf = RandomForestClassifier()\nclf = Linear...,/hs1592/adkkhn-krj-titanic,0.977052,76,Exploratory_Data_Analysis,commented
76486,10882513,6,"_ = clf.fit(x,y)",/hs1592/adkkhn-krj-titanic,0.999666,7,Model_Train,train_model
76487,10882513,7,pred_valid = clf.predict(x_valid)\naccuracy_sc...,/hs1592/adkkhn-krj-titanic,0.992896,48,Model_Evaluation,predict_on_test
76488,10882513,8,pred = clf.predict(x_test),/hs1592/adkkhn-krj-titanic,0.994685,48,Model_Evaluation,predict_on_test


In [6]:
distinct_kernel_links = kernels['kernel_link'].unique()
distinct_kernel_links[5]

'/rhythmcam/automl-search-best-titanic-model'

In [7]:
filtered_df = merged_df[merged_df['kernel_link'] == distinct_kernel_links[3]].sort_values('code_block_id', ascending=True)
filtered_df

,kernel_id,code_block_id,code_block,kernel_link,predicted_graph_vertex__probability,graph_vertex_id,graph_vertex_class,graph_vertex_subclass
117,21322805,0,import numpy as np\nimport pandas as pd\n\nfil...,/ryotaroyabe/notebook-titanic,0.999664,45,Data_Extraction,load_from_csv
118,21322805,1,"filename_test = ""/kaggle/input/titanic/test.cs...",/ryotaroyabe/notebook-titanic,0.999738,45,Data_Extraction,load_from_csv
119,21322805,2,"filename_submit=""../input/titanic/gender_submi...",/ryotaroyabe/notebook-titanic,0.999699,45,Data_Extraction,load_from_csv
120,21322805,3,"#大浜先生の質問\nanimals = pd.DataFrame({'Cows': [12,...",/ryotaroyabe/notebook-titanic,0.998248,12,Data_Transform,create_dataframe
121,21322805,4,"#Goatがmaxの時のTrue/Falesリスト\nprint(animals[""Goat...",/ryotaroyabe/notebook-titanic,0.956523,40,Exploratory_Data_Analysis,show_table_attributes
122,21322805,5,"data = pd.concat([train, test], sort=False)\ndata",/ryotaroyabe/notebook-titanic,0.999525,11,Data_Transform,concatenate
123,21322805,6,data.isnull().sum(),/ryotaroyabe/notebook-titanic,0.998980,39,Exploratory_Data_Analysis,count_missing_values
124,21322805,7,"data['Sex'].replace(['male', 'female'], [0, 1]...",/ryotaroyabe/notebook-titanic,0.798071,20,Data_Transform,categorify
125,21322805,8,data,/ryotaroyabe/notebook-titanic,0.999773,41,Exploratory_Data_Analysis,show_table
126,21322805,9,data['Embarked'].value_counts(),/ryotaroyabe/notebook-titanic,0.999534,72,Exploratory_Data_Analysis,count_values


In [ ]:
test_dataset_dir = "../../data/test_datasets"
merged_df.to_csv(f'{test_dataset_dir}/{top_comp}.csv', index=False)

#### Actual cell for saving the classified notebooks in a folder 

In [9]:
import sys
sys.path.insert(0, '../')
from utils.constants import BLANK_IPYNB_JSON
from utils.helper_functions import load_notebook
import json
import os
import tempfile 
from kaggle.api.kaggle_api_extended import ApiException
from tqdm import tqdm
test_dataset_dir = f"/home/ryounis/Documents/Zurich/PEACHLab/datascience-visualisation/data/datasets/{top_comp}/unclassified"
if not os.path.exists(test_dataset_dir): os.makedirs(test_dataset_dir)

FIRST_N_KERNELS = 20

notebooks = []
for kernel_link in tqdm(distinct_kernel_links[:FIRST_N_KERNELS]):
    with tempfile.TemporaryDirectory() as temp_dir:
        kernel_df = merged_df[merged_df['kernel_link'] == kernel_link].sort_values('code_block_id', ascending=True)
        try:
            kaggle.api.kernels_pull(kernel_link[1:], path=temp_dir, metadata=True)
            notebook_path = f"{temp_dir}/{kernel_link.split('/')[-1]}.ipynb"
            notebook = load_notebook(notebook_path)
        except ApiException as e:
            print(f"Error pulling kernel: {kernel_link}")
            continue
        
    code_cell_counter = 0
    for cell in notebook['cells']:
        if cell['cell_type'] == 'code' and len(cell['source']):
            row = kernel_df.iloc[code_cell_counter]
            code_cell_counter += 1

            cell["metadata"] = {
                **cell["metadata"],
                "class": row["graph_vertex_class"],
                "subclass": row["graph_vertex_subclass"],
                "subclass_id": int(row["graph_vertex_id"]),
                "predicted_subclass_probability": row["predicted_graph_vertex__probability"],
                "notebook_id": int(row['kernel_id']),
            }    
    notebook["metadata"]["labels"] = LABELS
    notebooks.append(notebook)
    filename = "_".join(kernel_link.split('/')[1:])
    with open(f"{test_dataset_dir}/{filename}.ipynb", "w") as f:json.dump(notebook, f)
    
print(f"Saved {len(notebooks)} notebooks to {test_dataset_dir}")

100%|██████████| 20/20 [00:12<00:00,  1.58it/s]

Saved 20 notebooks to /home/ryounis/Documents/Zurich/PEACHLab/datascience-visualisation/data/datasets/Titanic - Machine Learning from Disaster/unclassified


In [ ]:
from sklearn.cluster import HDBSCAN

clusterer = HDBSCAN(
    min_cluster_size=4,
    min_samples=2,
    cluster_selection_epsilon=.0,
    max_cluster_size=None,
    alpha=1.0
)

clusters = {}
for key, group in grouped_cells.items(): 
    X = np.array([cell["embedding"] for cell in group])
    
    if len(group) < 4:
        labels = [-1] * len(group)
    else:
        pca = PCA(n_components=2)
        pca.fit(X)
        embeddings_pca = pca.transform(X)
        clusterer.fit(embeddings_pca)
        labels = clusterer.labels_
        
    clusters[key] = {str(label): [] for label in set(labels)}
    for cell, label, embedding in zip(group, labels, embeddings_pca):
        clusters[key][str(label)].append({
            "cell": cell,
            "embedding": embedding
        })
    

In [ ]:
for key, elems in clusters.items():
    print(f"Class: {key}")
    for label, group in elems.items():
        print(f"    {label}: {len(group)}")
    print()

In [ ]:
import matplotlib.pyplot as plt

# Create a scatter plot of the embeddings
plt.figure(figsize=(10, 8))
for key, clusts in clusters.items():
    
    
    plt.scatter(
        x=embeddings_pca[:, 0],
        y=embeddings_pca[:, 1],
        cmap='viridis',
        alpha=0.5,
        label=key
    )
    plt.legend()
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.title('Clusters Visualization')
    plt.show()

In [ ]:
from utils.helper_functions import notebook_extract_code, notebook_add_class_labels

notebooks = []
for kernel_id in merged_df['kernel_id'].unique()[:20]:
    notebook_json = BLANK_IPYNB_JSON.copy()
    notebook_json['cells'] = []
    for row in merged_df[merged_df['kernel_id'] == kernel_id].iterrows():
        notebook_json['cells'].append({
            "cell_type": "code",
            "execution_count": None,
            "metadata": {
                "graph_vertex_id": row[1]["graph_vertex_id"],
                "predicted_graph_vertex__probability": row[1]["predicted_graph_vertex__probability"],
                "notebook_id": row[1]['kernel_id'],
            },
            "source": row[1]['code_block']
        })
    notebooks.append(notebook_json)
# notebook_code = notebook_extract_code(notebook_json)
# cell_labels = classifier.classify_notebook(notebook_code)
# notebook_json = notebook_add_class_labels(notebook_json, cell_labels)


In [ ]:
labeled_notebooks = []
notebook_cell_labels = []
for i, notebook_json in enumerate(notebooks):
    print(f"Notebook {i+1}/{len(notebooks)}", end='\r')
    
    notebook_code = notebook_extract_code(notebook_json)
    cell_labels = classifier.classify_notebook(notebook_code)
    notebook_json = notebook_add_class_labels(notebook_json, cell_labels)
    labeled_notebooks.append(notebook_json)
    notebook_cell_labels.append(cell_labels)

In [ ]:
embeddings = []
for notebook in notebook_cell_labels:
    for elem in notebook:
        embeddings.append(elem[1])
embeddings

In [ ]:
from sklearn.cluster import HDBSCAN

clusterer: HDBSCAN = HDBSCAN(
    min_cluster_size=4,                 # Minimum number of samples to form a cluster
    min_samples=2,                      # Minimum number of samples in a neighborhood to be considered as a core point
    cluster_selection_epsilon=0,     # If 2 clusters are less than epsilon apart, they get merged
)

clusterer.fit(embeddings)
for label in set(clusterer.labels_):
    print(f"Cluster {label}: {len([x for x in clusterer.labels_ if x == label])} cells")
clusterer.labels_

In [ ]:
counter = 0
for notebook in labeled_notebooks:
    for cell in notebook['cells']:
        cell['metadata']['cluster_label'] = clusterer.labels_[counter]
        counter += 1

In [ ]:
labeled_notebooks[0]["cells"]
true_labels = []
predicted_labels = []
for notebook in labeled_notebooks:
    for cell in notebook["cells"]:
        true_labels.append(cell["metadata"]["graph_vertex_id"])
        predicted_labels.append(cell["metadata"]["cluster_label"])

In [ ]:
from scipy.optimize import linear_sum_assignment
import numpy as np

def count_misclustered_elements(true_labels, predicted_labels):
    # Convert the labels to numpy arrays for easier manipulation
    true_labels = np.array(true_labels)
    predicted_labels = np.array(predicted_labels)
    
    unique_true_labels = np.unique(true_labels)
    unique_predicted_labels = np.unique(predicted_labels)
    
    cost_matrix = np.zeros((len(unique_true_labels), len(unique_predicted_labels)), dtype=int)
    
    for i, true_label in enumerate(unique_true_labels):
        for j, predicted_label in enumerate(unique_predicted_labels):
            cost_matrix[i, j] = np.sum((true_labels == true_label) & (predicted_labels != predicted_label))
    
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    
    misclustered_count = cost_matrix[row_ind, col_ind].sum()
    
    return misclustered_count

misclustered_count = count_misclustered_elements(true_labels, predicted_labels)
print(f"Number of misclustered elements: {misclustered_count}/{len(true_labels)}")
print(f"Score: {1 - misclustered_count/len(true_labels)} %")


In [ ]:
for notebook in labeled_notebooks:
    for cell in notebook['cells']:
        cell['metadata']['cluster_label'] = clusterer.labels_[counter]
        counter += 1